In [1]:
!pip install transformers datasets evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 r

## Load Dataset

In [12]:
from datasets import load_dataset

data = load_dataset("wmt/wmt19", "lt-en", split="train[:5000]")

train-00000-of-00002.parquet:   0%|          | 0.00/160M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/125M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/351k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2344893 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [13]:
data

Dataset({
    features: ['translation'],
    num_rows: 5000
})

In [14]:
datasets = data.train_test_split(test_size=0.2)

In [15]:
datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1000
    })
})

In [49]:
datasets['train'][0]['translation']['en']

'The natural disasters we witnessed in Europe this year come to prove the need for two substantial changes in the way in which we tackle these problems.'

In [29]:
datasets['test'][0]

{'translation': {'en': 'The enormous State aid that shipyards have received in the old Member States has bypassed the Gdańsk Shipyard.',
  'lt': 'Milžiniška valstybės pagalba, kurią laivų statyklos gavo iš senųjų valstybių narių, aplenkGdansko laivų statyklą.'}}

## Load Model and tokenizer

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## preprocess data

In [50]:
source_lang = "en"
target_lang = "lt"
prefix = "translate English to Latin: "


def preprocess_function(examples):
    inputs = [example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [51]:
tokenized_data = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Data Collator

In [52]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Evaluate matrics

In [53]:
import evaluate

metric = evaluate.load("sacrebleu")

In [54]:
from evaluate import load
import numpy as np

# Load BLEU metric
metric_bleu = load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Decode predictions (tokens -> sentences)
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Process and decode references (align at sentence level)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Align references and predictions at the sentence level
    decoded_labels = [[label] for label in decoded_labels]

    # Check alignment
    assert len(decoded_preds) == len(decoded_labels), \
        f"Number of predictions ({len(decoded_preds)}) does not match number of references ({len(decoded_labels)})"

    # Compute BLEU score
    bleu = metric_bleu.compute(predictions=decoded_preds, references=decoded_labels)

    return {
        "bleu": round(bleu["score"], 4),
    }


## Login HuggingFace

In [21]:
from huggingface_hub import notebook_login

notebook_login()

In [55]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer


training_args = Seq2SeqTrainingArguments(
    output_dir="en-lt-t5-small",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True, #change to bf16=True for XPU
    push_to_hub=True,
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [56]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 3.7100019454956055,
 'eval_model_preparation_time': 0.0056,
 'eval_bleu': 0.2769,
 'eval_runtime': 20.6327,
 'eval_samples_per_second': 48.467,
 'eval_steps_per_second': 3.053}

In [57]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Bleu
1,No log,3.200531,0.005600,2.059200
2,3.532000,3.101154,0.005600,1.887000
3,3.532000,3.034192,0.005600,1.931000
4,3.313200,2.982827,0.005600,1.885100
5,3.313200,2.941440,0.005600,1.852600
6,3.216500,2.909743,0.005600,1.819500
7,3.216500,2.885838,0.005600,1.851500
8,3.156200,2.869835,0.005600,1.800300
9,3.156200,2.860506,0.005600,1.780300
10,3.131200,2.856874,0.005600,1.799000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=2500, training_loss=3.269848291015625, metrics={'train_runtime': 537.2847, 'train_samples_per_second': 74.448, 'train_steps_per_second': 4.653, 'total_flos': 646481261887488.0, 'train_loss': 3.269848291015625, 'epoch': 10.0})

In [58]:
trainer.evaluate()

{'eval_loss': 2.856874465942383,
 'eval_model_preparation_time': 0.0056,
 'eval_bleu': 1.799,
 'eval_runtime': 18.9295,
 'eval_samples_per_second': 52.828,
 'eval_steps_per_second': 3.328,
 'epoch': 10.0}

In [59]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/osmanh/en-lt-t5-small/commit/4852e0655d06ad2f4b65d01b3e9624157790ebf0', commit_message='End of training', commit_description='', oid='4852e0655d06ad2f4b65d01b3e9624157790ebf0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/osmanh/en-lt-t5-small', endpoint='https://huggingface.co', repo_type='model', repo_id='osmanh/en-lt-t5-small'), pr_revision=None, pr_num=None)

## testing

In [60]:
text = "The enormous State aid that shipyards have received in the old Member States has bypassed the Gdańsk Shipyard."

In [61]:
from transformers import pipeline

translator = pipeline("translation_lt_to_en", model="osmanh/en-lt-t5-small")
translator(text)

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your input_length: 30 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'translation_text': 'Gdask sk i i'}]